In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
root="/amex/"
headers=pd.read_csv(root+"headers.csv")
headers = headers.columns
labels =pd.read_csv(root+"train_labels.csv")

In [3]:
preds = []
for r, d, f in os.walk(root+"test"):
    if r == "/amex/test":
        for file in f:
            if ".dt2.preds" in file :
                preds.append(os.path.join(r, file))
preds[len(preds)-1], len(preds)

('/amex/test/testfv.dt2.preds', 152)

In [24]:
preddf= pd.DataFrame()
for i in range(len(preds)):
    preddf = pd.concat([preddf, pd.read_csv(preds[i], names=['customer_ID', 'Label', 'Score_0', 'Score_1'], header=None)], ignore_index=True, axis=0)

In [25]:
preddf['Label'].min(), preddf['Label'].max()

(0.0, 1.0)

In [26]:
preddf = preddf.groupby(['customer_ID'], as_index=False)['Label'].mean() #.round() #.reset_index()
valids = pd.merge(samples, preddf, on='customer_ID', how='inner')
valids.rename(columns={'prediction': 'target'}, inplace=True)
valids.rename(columns={'Label': 'prediction'}, inplace=True)

valids['prediction'] = ( (valids[['prediction']] - valids[['prediction']].min()) / (valids[['prediction']].max() - valids[['prediction']].min()) ) * (1 - 0) + 0

print("matching 1's", valids[(valids['target'] == round(valids['prediction'])) & (valids['target'] == 1)].count()[0], valids[(valids['target'] == 1)].count()[0])
print("matching 0's", valids[(valids['target'] == round(valids['prediction'])) & (valids['target'] == 0)].count()[0], valids[(valids['target'] == 0)].count()[0])
print("matching", valids[valids['target'] == round(valids['prediction'])].count()[0], "total", valids.count()[0], "%", valids[valids['target'] == round(valids['prediction'])].count()[0]/valids.count()[0])

matching 1's 0 0
matching 0's 663351 924621
matching 663351 total 924621 % 0.7174301686853316


In [103]:
valids[valids['prediction']>= 0.000001]
valids['prediction'].min(), valids['prediction'].max()

(0.0, 1.0)

In [31]:
print("matching", valids[valids['target'] == round(valids['prediction'])].count()[0], "total", valids.count()[0], "%", valids[valids['target'] == round(valids['prediction'])].count()[0]/valids.count()[0])

matching 736896 total 924621 % 0.7969708669822554


In [56]:
valids[(valids['prediction'] <= 0.5)]

,customer_ID,target,prediction
753465,d09fd09ab7417840806298103d6f222371d27c596d2d11...,0,0.0


In [166]:
#!mv /amex/submission.csv submission11.csv

valids[['customer_ID', 'prediction']].to_csv('submission.csv', index=False)

In [2]:
800000/924621

0.8652193709638868

In [7]:
#samples = pd.read_csv(root + "sample_submission.csv")
#samples[samples['prediction'] == 0].count()[0]

In [4]:
#preddf= pd.DataFrame()
#for i in range(78):
#    preddf = pd.concat([preddf, pd.read_csv(preds[i], names=['customer_ID', 'Label'], header=None)], ignore_index=True, axis=0)

In [5]:
#preddf['Label'] = preddf['Label'].apply(lambda x: abs(x))
#preddf['Label'] = (preddf['Label'] / preddf['Label'].max())
#preddf['Label'] = preddf['Label'].apply(lambda x: 1 if x>1 else x)
#preddf['Label'] = preddf['Label'].apply(lambda x: 0 if x<0 else x)

#preddf = preddf.groupby(['customer_ID'], as_index=False)['Label'].mean() #.round() #.reset_index()

#pd.DataFrame(preddf)
#preddf['Label'] = preddf.Label.astype("int")
#preddf.head(1)

In [10]:
valids['prediction'].agg(['min', 'max'])
#731000/924621

min    0.0
max    1.0
Name: prediction, dtype: float64

In [24]:
#valids.loc[(valids['prediction']>0.1) & (valids['prediction']< 0.5), 'prediction'] = valids.loc[(valids['prediction']>0.1) & (valids['prediction']< 0.5), 'prediction'] * 0.9

#valids.loc[(valids['prediction']>0.1) & (valids['prediction']< 0.5), 'prediction']

df = (pd.concat([valids[['customer_ID', 'prediction']].rename(columns={'prediction':'target'}), valids[['customer_ID', 'prediction']]], axis='columns')).sort_values('prediction', ascending=False)
#df = (pd.concat([valids[['customer_ID', 'target']], valids[['customer_ID', 'prediction']]], axis='columns')).sort_values('prediction', ascending=False)

#df['correcpred'] = df['prediction'].apply(lambda x: 1 if x> 1 else (0 if x < 0 else x))

df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)

df['random'] = (df['weight'] / df['weight'].sum()).cumsum()

total_pos = (df['target'] * df['weight']).sum()

df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()

df['lorentz'] = df['cum_pos_found'] / total_pos

df['gini'] = (df['lorentz'] - df['random']) * df['weight']

#(df['correcpred'] * df['weight']).sum(), (df['prediction'] * df['weight']).sum()
df['gini'].sum()


#1782137.8168000695/303412.06746732915

#valids[valids['prediction']>1].max()
df#[df['target']>0.9]

,customer_ID,target,customer_ID,prediction,correcpred,weight,random,cum_pos_found,lorentz,gini
20136,059f588c612dbec6aaf6e3e53c33870e3e29ba4a43b35b...,1.000000,059f588c612dbec6aaf6e3e53c33870e3e29ba4a43b35b...,1.000000,1.000000,1,0.000001,1.000000,0.000003,2.122500e-06
420935,74ae4308d46ed012365fb05d219e6d977dd64ad4fd2c10...,0.994928,74ae4308d46ed012365fb05d219e6d977dd64ad4fd2c10...,0.994928,0.994928,1,0.000002,1.994928,0.000006,4.228749e-06
12019,035b6509f0e98768d1e91f6570acf20ffccd0649c23b83...,0.990917,035b6509f0e98768d1e91f6570acf20ffccd0649c23b83...,0.990917,0.990917,1,0.000003,2.985845,0.000010,6.322148e-06
109669,1e74bca35145daf997280ff28b1f125660d33c98580e81...,0.989633,1e74bca35145daf997280ff28b1f125660d33c98580e81...,0.989633,0.989633,1,0.000004,3.975478,0.000013,8.411432e-06
194376,35d85c566305bedfa007b65d014359c4a39bd6a213009f...,0.987661,35d85c566305bedfa007b65d014359c4a39bd6a213009f...,0.987661,0.987661,1,0.000005,4.963140,0.000016,1.049440e-05
...,...,...,...,...,...,...,...,...,...,...
178995,31a5cd081f3aa065914682b1915ef47f688ad4101333cf...,0.023784,31a5cd081f3aa065914682b1915ef47f688ad4101333cf...,0.023784,0.023784,1,0.999975,312109.635931,1.000000,2.468811e-05
488629,877db3eac69862b06cd555af7398c49d803fa4be62e683...,0.020438,877db3eac69862b06cd555af7398c49d803fa4be62e683...,0.020438,0.020438,1,0.999976,312109.656368,1.000000,2.367209e-05
180152,31f9ee6211f35ab2b3601c0428709de96f62924414c50d...,0.020392,31f9ee6211f35ab2b3601c0428709de96f62924414c50d...,0.020392,0.020392,1,0.999977,312109.676760,1.000000,2.265592e-05
410680,71e1ef894b8317979a53d43bda72601ba7b3d19f7354df...,0.017356,71e1ef894b8317979a53d43bda72601ba7b3d19f7354df...,0.017356,0.017356,1,0.999978,312109.694117,1.000000,2.163003e-05


In [16]:
int(0.04 * df['weight'].sum())
(df['target'] * df['weight']).sum()

0

In [76]:
tst = pd.read_csv("/amex/test/testaa", names=headers)

In [77]:
tst={}

In [29]:
#traindf = pd.read_csv("/amex/trainaa", names=headers)

traindf [['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']]

pd.DataFrame(traindf ['D_120'].unique())


,0
0,0.0
1,1.0
2,NaN


In [83]:
import numpy as np

traindf['D_120'] = traindf['D_120'].fillna(np.random.random(), inplace=True)

traindf['D_120']

0        None
1        None
2        None
3        None
4        None
         ... 
99994    None
99995    None
99996    None
99997    None
99998    None
Name: D_120, Length: 99999, dtype: object